In [1]:
import pandas as pd
import yfinance as yf
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
import xgboost as xgb
import backtrader as bt
import quantstats as qs
import pyfolio as pf
import json
from sklearn.ensemble import RandomForestClassifier
import os
from sklearn.metrics import confusion_matrix
import warnings
warnings.filterwarnings('ignore')
from fredapi import Fred
import numpy as np



/Users/pranuprakash/anaconda3/lib/python3.11/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; mutltipliers will not be applied to position notionals.
  warnings.warn(


In [2]:
start='2000-01-01'
end='2024-03-15'

In [3]:
indexpc = pd.read_csv('indexpc.csv', skiprows=2)
#indexpc.columns = ['Date', 'Call', 'Put', 'Total', 'PC_Ratio']

# Converting the 'Date' column to datetime format
indexpc['DATE'] = pd.to_datetime(indexpc['DATE'], errors='coerce')

# Filtering the data
indexpc = indexpc[(indexpc['DATE'] >= '2007-01-01') & (indexpc['DATE'] <= '2019-10-04')]

# Check the data types
indexpc


,DATE,CALL,PUT,TOTAL,P/C Ratio
41,2007-01-03,205724,309416,515140,1.50
42,2007-01-04,270783,293745,564528,1.08
43,2007-01-05,290570,352167,642737,1.21
44,2007-01-08,232352,273485,505837,1.18
45,2007-01-09,237668,276434,514102,1.16
...,...,...,...,...,...
3248,2019-09-30,582589,987408,1569997,1.69
3249,2019-10-01,617097,1050008,1667105,1.70
3250,2019-10-02,1376258,1782956,3159214,1.30
3251,2019-10-03,769129,1260434,2029563,1.64


In [4]:
dgs2_data = pd.read_csv('DGS2.csv')

# Convert the 'DATE' column to datetime and the 'DGS2' column to numeric
dgs2_data['DATE'] = pd.to_datetime(dgs2_data['DATE'])
dgs2_data['DGS2'] = pd.to_numeric(dgs2_data['DGS2'], errors='coerce')

# Calculate the daily percentage change of DGS2 values
dgs2_data['DGS2_pct_change'] = dgs2_data['DGS2'].pct_change()

# Filter the data for the specified date range (2000-01-01 to 2021-11-12)
dgs2_filtered = dgs2_data[(dgs2_data['DATE'] >= start) & (dgs2_data['DATE'] <= end)]

In [5]:
class Stock:
    def __init__(self, data):
        self.data = data
        self.label = []
    
    #preprocess
    #feature creation
    def factor(self):
        del self.data["Close"]
        self.data = self.data.fillna(method = "bfill")
        
        #return
        #>0,+1;<=0,-1
        self.data['label']=self.data.rolling(2).apply(lambda x:x.iloc[1]>x.iloc[0])['Adj Close']
        self.data['label']=self.data.label.shift(-1)

        # Volume Change (daily percentage change)
        self.data['volume_pct_change'] = self.data['Volume'].pct_change()

        # Volume Moving Averages
        self.data['vol_ma_5'] = self.data['Volume'].rolling(window=5).mean()
        #self.data['vol_ma_10'] = self.data['Volume'].rolling(window=10).mean()
        self.data['vol_ma_20'] = self.data['Volume'].rolling(window=20).mean()  
        self.data['vol_ma_50'] = self.data['Volume'].rolling(window=50).mean()  
        self.data['vol_ma_200'] = self.data['Volume'].rolling(window=200).mean()  

        #moving average
        self.data["ma_5"] = self.data["Adj Close"].rolling(window = 5).mean()
        self.data["ma_10"] = self.data["Adj Close"].rolling(window = 10).mean()
        self.data["ma_20"] = self.data["Adj Close"].rolling(window = 20).mean()
        self.data["ma_50"] = self.data["Adj Close"].rolling(window = 50).mean()
        self.data["ma_200"] = self.data["Adj Close"].rolling(window = 200).mean()

        
        #self.merge_pc_ratio(indexpc)
        self.add_volume_oscillator()
        self.add_relative_volume()
        self.add_volume_relative_to_ma()
        self.add_price_volume_trend()
        self.add_vix_feature()
        self.add_spy_vix_ratio_feature()
        self.add_rsi_feature()
        self.add_bollinger_bands_feature()
        self.add_cci_feature()
        
        
        self.add_10yr_bond_yield_change_feature()
        self.add_yield_spread_feature(dgs2_filtered)
        self.add_atr_feature()

        
        #self.add_volume_spikes()
        self.add_stochastic_oscillator()
        self.add_spy_iwm_ratio_feature()
        self.add_spy_qqq_ratio_feature()
        self.add_spy_dia_ratio_feature()
        self.add_dxy_feature()
        self.add_obv_feature()
        #self.create_combined_conditions()


        self.add_momentum_feature_5d(window=5)
        
        #self.add_momentum_feature_10d(window=10)
        #self.add_momentum_feature_20d(window=20)
        #self.add_momentum_feature_50d(window=50)
        #self.add_momentum_feature_200d(window=200)


        #self.add_volatility_feature_short(window=21)
        self.add_volatility_feature_long(window=252)

        self.add_atr_feature()
        self.add_momentum_volatility_feature()
        
        #del self.data["SPY_pct_change"]
        #del self.data["IWM_pct_change"]
        #self.add_2yr_treasury_yield_change_feature(dgs2_filtered)  # Correctly pass dgs2_filtered as an argument

        
        self.data = self.data.dropna(how = "any")
        self.label = list(self.data["label"])
        del self.data["label"]
        del self.data["Adj Close"]
        del self.data["Open"]
        del self.data["High"]
        del self.data["Low"]
        
        del self.data['avg_volume']
        del self.data['vix_pct_change']
        del self.data['vix']
        #del self.data['macd_signal']
        #del self.data['is_uptrend']
        #del self.data['is_downtrend']
        #del self.data['is_vix_high'] 
        #del self.data['is_vix_moderate']
        #del self.data['is_vix_low']
        #del self.data['macd']
        print(self.data.columns)
        print(self.data.tail())


    def add_reversal_feature(self, window=5):
        """
        Add reversal feature based on the negative of momentum.
        A simple reversal feature could be the negative price change over a given window.
        """
        self.data['reversal_feature'] = -self.data['Adj Close'].pct_change(periods=window)

        # Now, make sure to forward fill any NaN values that were generated
        self.data['reversal_feature'] = self.data['reversal_feature'].fillna(method="ffill")

        # Optionally, rank the reversal features if you are considering a cross-sectional strategy
        # self.data['reversal_rank'] = self.data['reversal_feature'].rank(pct=True)

        # Normalize the reversal feature so that it's on the same scale as your other features
        self.data['reversal_feature'] = (self.data['reversal_feature'] - self.data['reversal_feature'].mean()) / self.data['reversal_feature'].std()

    
    def price_acceleration(self):
        lagged_day = 5
        self.data['lagged_data_1'] = self.data['Adj Close'].shift(lagged_day)
        self.data['lagged_data_2'] = self.data['Adj Close'].shift(lagged_day * 2)
        self.data['price_rate_change_1'] = (self.data['Adj Close'] - self.data['lagged_data_1']) / lagged_day
        self.data['price_rate_change_2'] = (self.data['lagged_data_1'] - self.data['lagged_data_2']) / lagged_day
        self.data['price_acceleration'] = (self.data['price_rate_change_1'] - self.data['price_rate_change_2']) / lagged_day
        del self.data['lagged_data_1']
        del self.data['lagged_data_2']
        del self.data['price_rate_change_1']
        del self.data['price_rate_change_2']

    def percent_off_52_weeks_high(self):
        one_year_high = self.data['Adj Close'].rolling(window=252).max()
        self.data['percent_off_52_weeks_high'] = (one_year_high - self.data['Adj Close']) / one_year_high

    
    def add_atr_feature(self, window=14):
        """Add Average True Range (ATR) feature."""
        high_low = self.data['High'] - self.data['Low']
        high_close = np.abs(self.data['High'] - self.data['Adj Close'].shift())
        low_close = np.abs(self.data['Low'] - self.data['Adj Close'].shift())

        true_ranges = np.maximum(high_low, high_close, low_close)
        self.data['atr'] = true_ranges.rolling(window=window).mean()

    def add_momentum_volatility_feature(self, momentum_window=14, vol_window=21):
        """Add Momentum-Based Volatility feature."""
        momentum = self.data['Adj Close'].pct_change(periods=momentum_window)
        self.data['momentum_volatility'] = momentum.rolling(window=vol_window).std()
        

        
    def add_volatility_feature_short(self, window=21):  # 21 trading days roughly equals 1 month
        """Add Volatility feature calculated as the rolling standard deviation of daily returns."""
        # Calculate daily returns
        daily_returns = self.data['Adj Close'].pct_change()
        
        # Calculate the rolling standard deviation of daily returns
        self.data['volatility_21d'] = daily_returns.rolling(window=window).std() * np.sqrt(window)


    def add_volatility_feature_long(self, window=252):  # 21 trading days roughly equals 1 month
        """Add Volatility feature calculated as the rolling standard deviation of daily returns."""
        # Calculate daily returns
        daily_returns = self.data['Adj Close'].pct_change()
        
        # Calculate the rolling standard deviation of daily returns
        self.data['volatility_252d'] = daily_returns.rolling(window=window).std() * np.sqrt(window)
    
    def merge_pc_ratio(self, pc_ratio_data):
        # Ensure 'DATE' is in datetime format and set it as the index
        if 'DATE' in pc_ratio_data.columns:
            pc_ratio_data['DATE'] = pd.to_datetime(pc_ratio_data['DATE'])
            pc_ratio_data.set_index('DATE', inplace=True)
    
        # Calculate the P/C Ratio change (you can use pct_change() for percentage change)
        pc_ratio_data['PC_Ratio_Change'] = pc_ratio_data['P/C Ratio'].pct_change()
    
        # Merge the PC_Ratio_Change into self.data
        self.data = self.data.join(pc_ratio_data['PC_Ratio_Change'], how='left')
    
        # Handle any infinite values and fill missing values
        self.data['PC_Ratio_Change'].replace([np.inf, -np.inf], np.nan, inplace=True)
        self.data['PC_Ratio_Change'] = self.data['PC_Ratio_Change'].fillna(method="bfill")
    

    def add_momentum_feature_5d(self, window=5):
        """Add Momentum feature."""
        self.data['momentum_5d'] = self.data['Adj Close'].pct_change(periods=window).shift(-window)

    def add_momentum_feature_10d(self, window=10):
        """Add Momentum feature."""
        self.data['momentum_10d'] = self.data['Adj Close'].pct_change(periods=window).shift(-window)

    def add_momentum_feature_20d(self, window=20):
        """Add Momentum feature."""
        self.data['momentum_10d'] = self.data['Adj Close'].pct_change(periods=window).shift(-window)

    def add_momentum_feature_50d(self, window=50):
        """Add Momentum feature."""
        self.data['momentum_10d'] = self.data['Adj Close'].pct_change(periods=window).shift(-window)

    def add_momentum_feature_200d(self, window=200):
        """Add Momentum feature."""
        self.data['momentum_10d'] = self.data['Adj Close'].pct_change(periods=window).shift(-window)
               
    def add_volume_oscillator(self):
            """Add Volume Oscillator feature."""
            short_term = 5
            long_term = 10
            
            self.data['volume_oscillator'] = self.data['Volume'].rolling(window=short_term).mean() - self.data['Volume'].rolling(window=long_term).mean()

    def add_relative_volume(self, comparison_period=20):
        """Add Relative Volume feature."""
        # Calculate the average volume over the specified comparison period
        self.data['avg_volume'] = self.data['Volume'].rolling(window=comparison_period).mean()

        # Calculate Relative Volume
        self.data['relative_volume'] = self.data['Volume'] / self.data['avg_volume']

    def add_volume_relative_to_ma(self, period=50):
        """Add Volume Relative to Moving Average."""
        self.data['vol_relative_to_ma'] = self.data['Volume'] / self.data['Volume'].rolling(window=period).mean()

    def add_volume_spikes(self, threshold=2):
        """Add Volume Spikes."""
        self.data['vol_spike'] = self.data['Volume'] > self.data['Volume'].rolling(window=50).mean() * threshold

    def add_price_volume_trend(self):
        """Add Price-Volume Trend."""
        self.data['pvt'] = (self.data['Volume'] * self.data['Adj Close'].diff()).cumsum()

    def download_vix(self):
        """Download VIX data for the same date range as the stock data."""
        start_date = self.data.index.min().strftime('%Y-%m-%d')
        end_date = self.data.index.max().strftime('%Y-%m-%d')
        self.vix_data = yf.download("^VIX", start=start, end=end)['Close']

    def add_vix_feature(self):
        """Add VIX as a feature along with its percentage change."""
        self.download_vix()  # Download VIX data

        # Merge raw VIX data into the stock data
        self.data['vix'] = self.vix_data.reindex(self.data.index, method='bfill')

        # Calculate the percentage change in VIX
        self.data['vix_pct_change'] = self.data['vix'].pct_change()

        # Handle any missing values
        self.data['vix_pct_change'].fillna(method='bfill', inplace=True)


    def download_spy(self):
        """Download SPY data for the same date range as the stock data."""
        start_date = self.data.index.min().strftime('%Y-%m-%d')
        end_date = self.data.index.max().strftime('%Y-%m-%d')
        self.spy_data = yf.download("SPY", start=start, end=end)['Adj Close']

    def IWM_moving_beta_change(self):
        """Calculate the change in the 30-day rolling beta of IWM relative to SPX."""
        # Download SPX data
        start_date = self.data.index.min().strftime('%Y-%m-%d')
        end_date = self.data.index.max().strftime('%Y-%m-%d')
        spy_data = yf.download("SPY", start=start_date, end=end_date)['Adj Close']
        iwm_data= yf.download("IWM", start=start_date, end=end_date)['Adj Close']
        # Calculate returns
        iwm_returns = iwm_data.pct_change()
        spy_returns = spy_data.pct_change()

        # Calculate 30-day rolling beta
        covariance = iwm_returns.rolling(window=30).cov(spy_returns)
        variance = spy_returns.rolling(window=30).var()
        rolling_beta = covariance / variance

        # Calculate the change in rolling beta
        change_in_rolling_beta = rolling_beta.diff()

        # Store the change in rolling beta in the class
        self.data['Change_in_Rolling_Beta_IWM'] = change_in_rolling_beta.shift(5)
        

    def add_spy_vix_ratio_feature(self):
        """Add feature of SPY price change / VIX price change."""
        self.download_spy()  # Download SPY data
        self.download_vix()  # Download VIX data

        # Calculate daily percentage change for SPY and VIX
        spy_pct_change = self.spy_data.pct_change()
        vix_pct_change = self.vix_data.pct_change()
        
        # Calculate the ratio of SPY change to VIX change
        self.data['SPY_VIX_ratio'] = spy_pct_change/ vix_pct_change
        self.data['SPY_VIX_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
        self.data['SPY_VIX_ratio'] = self.data['SPY_VIX_ratio'].fillna(method = "bfill")

    def download_etf_data(self, ticker, column_name):
        """Download ETF data for the same date range as the stock data."""
        start_date = self.data.index.min().strftime('%Y-%m-%d')
        end_date = self.data.index.max().strftime('%Y-%m-%d')
        etf_data = yf.download(ticker, start=start, end=end)['Adj Close']
        etf_pct_change = etf_data.pct_change()
        self.data[column_name] = etf_pct_change

    def add_spy_iwm_ratio_feature(self):
        """Add feature of SPY price change / IWM price change."""
        self.download_etf_data("SPY", "SPY_pct_change")
        self.download_etf_data("IWM", "IWM_pct_change")

        # Calculate the ratio of SPY change to IWM change
        self.data['SPY_IWM_ratio'] = self.data['SPY_pct_change'] / self.data['IWM_pct_change']
        self.data['SPY_IWM_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
        self.data['SPY_IWM_ratio'] = self.data['SPY_VIX_ratio'].fillna(method = "bfill")
    def add_spy_qqq_ratio_feature(self):
        """Add feature of SPY price change / QQQ price change."""
        self.download_etf_data("SPY", "SPY_pct_change")
        self.download_etf_data("QQQ", "QQQ_pct_change")
        self.data['SPY_QQQ_ratio'] = self.data['SPY_pct_change'] / self.data['QQQ_pct_change']
        self.data['SPY_QQQ_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
        self.data['SPY_QQQ_ratio'] = self.data['SPY_VIX_ratio'].fillna(method = "bfill")
    def add_spy_dia_ratio_feature(self):
        """Add feature of SPY price change / DIA price change."""
        self.download_etf_data("SPY", "SPY_pct_change")
        self.download_etf_data("DIA", "DIA_pct_change")
        self.data['SPY_DIA_ratio'] = self.data['SPY_pct_change'] / self.data['DIA_pct_change']
        self.data['SPY_DIA_ratio'].replace([np.inf, -np.inf], np.nan, inplace=True)
        self.data['SPY_DIA_ratio'] = self.data['SPY_VIX_ratio'].fillna(method = "bfill")

    def add_dxy_feature(self):
        """Add feature of DXY (US Dollar Index) daily percentage change."""
    # Download DXY data for the same date range as the stock data

        dxy_data = yf.download("DX-Y.NYB", start=start, end=end)['Adj Close']
        
        # Calculate daily percentage change for DXY
        dxy_pct_change = dxy_data.pct_change()
        
        # Add the DXY daily percentage change to the stock data DataFrame
        self.data['DXY_pct_change'] = dxy_pct_change
        self.data['DXY_pct_change'].replace([np.inf, -np.inf], np.nan, inplace=True)
        self.data['DXY_pct_change'] = self.data['DXY_pct_change'].fillna(method="bfill")
    def download_bond_yield_data(self, ticker, column_name):
        """Download bond yield data for the same date range as the stock data."""

        bond_data = yf.download(ticker, start=start, end=end)['Adj Close']
        bond_pct_change = bond_data.pct_change()
        self.data[column_name] = bond_pct_change

    def add_2yr_treasury_yield_change_feature(self, dgs2_data):
        """
        Add the 2-year Treasury yield percentage change as a feature to the stock data.
        
        Args:
        dgs2_data (DataFrame): DataFrame containing the DGS2 data with 'DATE' and 'DGS2_pct_change' columns.
        """
        # Check if 'DATE' column exists in dgs2_data
        if 'DATE' in dgs2_data.columns:
            # If 'DATE' column exists, ensure it's in datetime format and set it as the index
            dgs2_data['DATE'] = pd.to_datetime(dgs2_data['DATE'])
            dgs2_data.set_index('DATE', inplace=True)
        
        # Merge the DGS2_pct_change into the stock data
        self.data = self.data.join(dgs2_data['DGS2_pct_change'], how='left')

        # Handle any infinite values and fill missing values
        self.data['DGS2_pct_change'].replace([np.inf, -np.inf], np.nan, inplace=True)
        self.data['DGS2_pct_change'] = self.data['DGS2_pct_change'].fillna(method="bfill")
    
    def add_10yr_bond_yield_change_feature(self):
        """Add change of 10-year bond yield as a feature and remove the original yield data."""
        self.download_bond_yield_data("^TNX", "10yr_bond_yield")
        self.data['10yr_bond_yield_change'] = self.data['10yr_bond_yield'].pct_change() * 100
    
        # Replace inf/-inf with NaN
        self.data.replace([np.inf, -np.inf], np.nan, inplace=True)
    
        # Handle NaN values, for example, by forward filling and then back filling
        self.data['10yr_bond_yield_change'].fillna(method='ffill', inplace=True)
        self.data['10yr_bond_yield_change'].fillna(method='bfill', inplace=True)
        
        # Drop the original "10yr_bond_yield" column
        self.data.drop(columns=['10yr_bond_yield'], inplace=True)

    def add_yield_spread_feature(self, dgs2_data):
        """
        Add the feature representing the difference between 2-year and 10-year Treasury bond yields.
        
        Args:
        dgs2_data (DataFrame): DataFrame containing the local DGS2 data.
        """
        # Download 10-year Treasury yield data
        dgs10_data = yf.download("^TNX", start="2000-01-01", end="2023-12-20")['Adj Close']

        # Ensure DGS2 data is in the correct format
        if 'DATE' in dgs2_data.columns:
            dgs2_data.set_index('DATE', inplace=True)
        dgs2_data.index = pd.to_datetime(dgs2_data.index)

        # Align the DGS10 data with DGS2 data dates
        dgs10_aligned = dgs10_data.reindex(dgs2_data.index, method='bfill')

        # Calculate the yield spread
        yield_spread = dgs10_aligned - dgs2_data['DGS2']

        # Add the yield spread to the stock data
        self.data['yield_spread'] = yield_spread

        # Handle missing values
        self.data['yield_spread'].fillna(method='bfill', inplace=True)

    def add_rsi_feature(self, window=14):
        """Add Relative Strength Index (RSI) feature."""
        delta = self.data['Adj Close'].diff()
        gain = (delta.where(delta > 0, 0)).rolling(window=window).mean()
        loss = (-delta.where(delta < 0, 0)).rolling(window=window).mean()

        rs = gain / loss
        self.data['rsi'] = 100 - (100 / (1 + rs))

    def add_bollinger_bands_feature(self, window=20, num_std=2):
        """Add Bollinger Bands feature."""
        rolling_mean = self.data['Adj Close'].rolling(window=window).mean()
        rolling_std = self.data['Adj Close'].rolling(window=window).std()

        self.data['bollinger_upper'] = rolling_mean + (rolling_std * num_std)
        self.data['bollinger_lower'] = rolling_mean - (rolling_std * num_std)

    def add_atr_feature(self, window=14):
        """Add Average True Range (ATR) feature."""
        high_low = self.data['High'] - self.data['Low']
        high_close = np.abs(self.data['High'] - self.data['Adj Close'].shift())
        low_close = np.abs(self.data['Low'] - self.data['Adj Close'].shift())

        ranges = pd.concat([high_low, high_close, low_close], axis=1)
        true_range = np.max(ranges, axis=1)
        self.data['atr'] = true_range.rolling(window=window).mean()

    def add_stochastic_oscillator(self, k_window=14, d_window=3):
        """Add Stochastic Oscillator feature."""
        min_low = self.data['Low'].rolling(window=k_window).min()
        max_high = self.data['High'].rolling(window=k_window).max()

        self.data['%K'] = 100 * ((self.data['Adj Close'] - min_low) / (max_high - min_low))
        self.data['%D'] = self.data['%K'].rolling(window=d_window).mean()

    def add_cci_feature(self, window=20):
        """Add Commodity Channel Index (CCI) feature."""
        tp = (self.data['High'] + self.data['Low'] + self.data['Adj Close']) / 3
        cci = (tp - tp.rolling(window=window).mean()) / (0.015 * tp.rolling(window=window).std())
        self.data['cci'] = cci

    def add_obv_feature(self):
        """Add On-Balance Volume (OBV) feature."""
        obv = (np.sign(self.data['Adj Close'].diff()) * self.data['Volume']).fillna(0).cumsum()
        self.data['obv'] = obv

    def calculate_macd(self):
        # Calculate MACD
        ema_short = self.data['Adj Close'].ewm(span=12, adjust=False, min_periods=12).mean()
        ema_long = self.data['Adj Close'].ewm(span=26, adjust=False, min_periods=26).mean()
        self.data['macd'] = ema_short - ema_long
        self.data['macd_signal'] = self.data['macd'].ewm(span=9, adjust=False, min_periods=9).mean()
        
    def classify_trend(self):
        self.calculate_macd()
        self.data['is_uptrend'] = (self.data['macd'] > self.data['macd_signal']).astype(int)
        self.data['is_downtrend'] = (self.data['macd'] < self.data['macd_signal']).astype(int)

    def add_vix_categories(self):
        vix_80_percentile = self.data['vix'].quantile(0.8)
        vix_40_percentile = self.data['vix'].quantile(0.4)
        self.data['is_vix_high'] = (self.data['vix'] > 30).astype(int)
        self.data['is_vix_moderate'] = ((self.data['vix'] >= 20) & (self.data['vix'] <= 30)).astype(int)
        self.data['is_vix_low'] = (self.data['vix'] < 20).astype(int)
    
    def create_combined_conditions(self):
        self.classify_trend()
        self.add_vix_categories()
    
        self.data['up_high'] = (self.data['is_uptrend'] & self.data['is_vix_high']).astype(int)
        self.data['down_high'] = (self.data['is_downtrend'] & self.data['is_vix_high']).astype(int)
        self.data['up_moderate'] = (self.data['is_uptrend'] & self.data['is_vix_moderate']).astype(int)
        self.data['down_moderate'] = (self.data['is_downtrend'] & self.data['is_vix_moderate']).astype(int)
        self.data['up_low'] = (self.data['is_uptrend'] & self.data['is_vix_low']).astype(int)
        self.data['down_low'] = (self.data['is_downtrend'] & self.data['is_vix_low']).astype(int)



    #standardize data
    def standardize(self):
        scaler = StandardScaler()      
        self.data = scaler.fit_transform(self.data)

    
    #normalize data
    def normalize(self):
        scaler = MinMaxScaler()      
        self.data = scaler.fit_transform(self.data)
  
    # Assuming self.data is a pandas DataFrame and self.label is the target variable
    


In [6]:
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
import xgboost as xgb

# Assuming Stock class and list1 are defined as before

# Lists to store combined features and labels for all stocks
all_features_xgb = []
all_labels_xgb = []

# Download and preprocess data for each ticker in the list
yf_data = yf.download("SPY", start=start, end=end)
stock = Stock(yf_data)
stock.factor()        # Feature creation and preprocessing
stock.standardize()   # Standardizing the data
stock.normalize()     # Normalizing the data

# Append the features and labels to the respective lists
all_features_xgb.append(pd.DataFrame(stock.data))
all_labels_xgb.extend(stock.label)

X = pd.concat(all_features_xgb, ignore_index=True)
y = pd.Series(all_labels_xgb)

# Set a fixed random state for reproducibility
fixed_random_state = 42

# Splitting the dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,shuffle=False)

# Initialize an XGBoost classifier with adjusted parameters
xgb_model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        max_depth=3,
        n_estimators=100,
        gamma=0.1,
        subsample=0.6,
        colsample_bytree=0.8,
        learning_rate=0.01,
        random_state=fixed_random_state
    )


# Train the model
xgb_model.fit(X_train, y_train)

# Make predictions
pred_y = xgb_model.predict(X_test)

# Calculating accuracy and precision
accuracy_xgb = accuracy_score(y_test, pred_y)
precision_xgb = precision_score(y_test, pred_y)
print("accuracy_xgb: ", accuracy_xgb)
print("precision_xgb: ", precision_xgb)
cm = confusion_matrix(y_test, pred_y)
print("Confusion Matrix:\n", cm)

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


Index(['Volume', 'volume_pct_change', 'vol_ma_5', 'vol_ma_20', 'vol_ma_50',
       'vol_ma_200', 'ma_5', 'ma_10', 'ma_20', 'ma_50', 'ma_200',
       'volume_oscillator', 'relative_volume', 'vol_relative_to_ma', 'pvt',
       'SPY_VIX_ratio', 'rsi', 'bollinger_upper', 'bollinger_lower', 'cci',
       '10yr_bond_yield_change', 'yield_spread', 'atr', '%K', '%D',
       'SPY_pct_change', 'IWM_pct_change', 'SPY_IWM_ratio', 'QQQ_pct_change',
       'SPY_QQQ_ratio', 'DIA_pct_change', 'SPY_DIA_ratio', 'DXY_pct_change',
       'obv', 'momentum_5d', 'volatility_252d', 'momentum_volatility'],
      dtype='object')
               Volume  volume_pct_change    vol_ma_5   vol_ma_20   vol_ma_50  \
Date                                                                           
2023-12-13   93278000           0.365158  75336820.0  68404525.0  79773644.0   
2023-12-14  119026000           0.276035  85742940.0  70489445.0  80405104.0   
2023-12-15  141319300           0.187298  97390620.0  74222120.0  818

In [7]:
from sklearn.model_selection import GridSearchCV

param_grid = {
    'max_depth': [3, 4, 5],
    'n_estimators': [100, 150, 200],
    'learning_rate': [0.01, 0.05, 0.1],
    'gamma': [0.1, 0.2, 0.3],
    'subsample': [0.6, 0.7, 0.8],
    'colsample_bytree': [0.6, 0.7, 0.8]
}
xgb_model = xgb.XGBClassifier(
    objective='binary:logistic',
    eval_metric='logloss',
    random_state=fixed_random_state
)

grid_search = GridSearchCV(estimator=xgb_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)
grid_search.fit(X_train, y_train)
best_model = grid_search.best_estimator_
pred_y = best_model.predict(X_test)

# Evaluate the model
accuracy = accuracy_score(y_test, pred_y)
precision = precision_score(y_test, pred_y)
print("Best Model Accuracy:", accuracy)
print("Best Model Precision:", precision)
print("Best Model Parameters:", grid_search.best_params_)


Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   0.4s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   0.5s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   0.5s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   0.5s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.6; total time=   0.5s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.5s
[CV] END colsample_bytree=0.6, gamma=0.1, learning_rate=0.01, max_depth=3, n_estimators=100, subsample=0.7; total time=   0.5s
[CV] END colsample_bytree=0.6, gamma=0.1, learn

KeyboardInterrupt: 

In [ ]:
import numpy as np

# Get feature importances
importances = xgb_model.feature_importances_

# Get different thresholds
thresholds = np.sort(np.unique(importances))

# Dictionary to store results
results = {}

for thresh in thresholds:
    # Select features using threshold
    selection = X_train.columns[importances >= thresh]
    
    # Select training and testing data based on selected features
    select_X_train = X_train[selection]
    select_X_test = X_test[selection]
    
    # Train model
    selection_model = xgb.XGBClassifier(
        objective='binary:logistic',
        eval_metric='logloss',
        max_depth=3,
        n_estimators=100,
        gamma=0.1,
        subsample=0.6,
        colsample_bytree=0.8,
        learning_rate=0.01,
        random_state=fixed_random_state
    )

    selection_model.fit(select_X_train, y_train)
    
    # Evaluate model
    select_pred_y = selection_model.predict(select_X_test)
    select_accuracy = accuracy_score(y_test, select_pred_y)
    
    # Store results
    results[thresh] = {'features': len(selection), 'accuracy': select_accuracy}

# Initialize variables to track the highest accuracy
highest_accuracy = 0
highest_accuracy_threshold = 0
highest_accuracy_features = 0

# Print details for all thresholds
print("All Thresholds with Corresponding Number of Features and Accuracy:")
for thresh, data in results.items():
    print(f"Threshold: {thresh:.4f}, Number of Features: {data['features']}, Accuracy: {data['accuracy']:.4f}")

    # Update the highest accuracy and its details
    if data['accuracy'] > highest_accuracy:
        highest_accuracy = data['accuracy']
        highest_accuracy_threshold = thresh
        highest_accuracy_features = data['features']

# Print the details for the highest accuracy
print("\nHighest Accuracy:")
print(f"Threshold: {highest_accuracy_threshold:.4f}, Number of Features: {highest_accuracy_features}, Accuracy: {highest_accuracy:.4f}")


Search in more detailed way

In [50]:
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.model_selection import TimeSeriesSplit

# Define the space of hyperparameters to search
space = {
    'max_depth': hp.choice('max_depth', range(0, 10)),
    'n_estimators': hp.choice('n_estimators', range(0, 300)),
    'learning_rate': hp.uniform('learning_rate', 0.01, 0.1),
    'gamma': hp.uniform('gamma', 0.0, 0.5),
    'subsample': hp.uniform('subsample', 0.1, 0.9),
    'colsample_bytree': hp.uniform('colsample_bytree', 0.1, 0.9)
}

def objective(params):
    params = {'max_depth': int(params['max_depth']),
              'n_estimators': int(params['n_estimators']),
              'learning_rate': params['learning_rate'],
              'gamma': params['gamma'],
              'subsample': params['subsample'],
              'colsample_bytree': params['colsample_bytree']}
    
    xgb_model = xgb.XGBClassifier(**params)
    
    # Use TimeSeriesSplit for cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    best_score = cross_val_score(xgb_model, X_train, y_train, scoring='accuracy', cv=tscv).mean()
    return {'loss': -best_score, 'status': STATUS_OK}
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

print("Best: ", best)

100%|██████| 100/100 [03:09<00:00,  1.90s/trial, best loss: -0.6405194805194806]
Best:  {'colsample_bytree': 0.39597682865661404, 'gamma': 0.23000324714086606, 'learning_rate': 0.014346501379076927, 'max_depth': 1, 'n_estimators': 137, 'subsample': 0.8389238792570485}


In [52]:
# Best hyperparameters with indices
best_indices = {'colsample_bytree': 0.39597682865661404, 'gamma': 0.23000324714086606, 'learning_rate': 0.014346501379076927, 'max_depth': 1, 'n_estimators': 137, 'subsample': 0.8389238792570485}
# Convert indices to actual values
max_depth_values = range(0, 10)  # The range you defined in the hyperopt space
n_estimators_values = range(0, 300)  # The range you defined in the hyperopt space

# Actual best hyperparameters
best_hyperparameters = {
    'colsample_bytree': best_indices['colsample_bytree'],
    'gamma': best_indices['gamma'],
    'learning_rate': best_indices['learning_rate'],
    'max_depth': max_depth_values[best_indices['max_depth']],
    'n_estimators': n_estimators_values[best_indices['n_estimators']],
    'subsample': best_indices['subsample']
}

print("Best Hyperparameters:", best_hyperparameters)


Best Hyperparameters: {'colsample_bytree': 0.39597682865661404, 'gamma': 0.23000324714086606, 'learning_rate': 0.014346501379076927, 'max_depth': 1, 'n_estimators': 137, 'subsample': 0.8389238792570485}


In [53]:
import numpy as np
from sklearn.metrics import accuracy_score
import xgboost as xgb

# Assuming best_params is already defined and adjusted as per previous instructions
best_params = {'colsample_bytree': 0.39597682865661404, 'gamma': 0.23000324714086606, 'learning_rate': 0.014346501379076927, 'max_depth': 1, 'n_estimators': 137, 'subsample': 0.8389238792570485}


xgb_model.fit(X_train, y_train)  # Ensure X_train and y_train are defined
importances = xgb_model.feature_importances_

# Get different thresholds
thresholds = np.sort(np.unique(importances))

# Dictionary to store results
results = {}

for thresh in thresholds:
    # Select features using threshold
    selection = X_train.columns[importances >= thresh]
    
    # Select training and testing data based on selected features
    select_X_train = X_train[selection]
    select_X_test = X_test[selection]
    
    # Train model using the best hyperparameters
    selection_model = xgb.XGBClassifier(**best_params)
    selection_model.fit(select_X_train, y_train)
    
    # Evaluate model
    select_pred_y = selection_model.predict(select_X_test)
    select_accuracy = accuracy_score(y_test, select_pred_y)
    
    # Store results
    results[thresh] = {'features': len(selection), 'accuracy': select_accuracy}

# Initialize variables to track the highest accuracy
highest_accuracy = 0
highest_accuracy_threshold = 0
highest_accuracy_features = 0

# Print details for all thresholds
print("All Thresholds with Corresponding Number of Features and Accuracy:")
for thresh, data in results.items():
    print(f"Threshold: {thresh:.4f}, Number of Features: {data['features']}, Accuracy: {data['accuracy']:.4f}")

    # Update the highest accuracy and its details
    if data['accuracy'] > highest_accuracy:
        highest_accuracy = data['accuracy']
        highest_accuracy_threshold = thresh
        highest_accuracy_features = data['features']

# Print the details for the highest accuracy
print("\nHighest Accuracy:")
print(f"Threshold: {highest_accuracy_threshold:.4f}, Number of Features: {highest_accuracy_features}, Accuracy: {highest_accuracy:.4f}")


All Thresholds with Corresponding Number of Features and Accuracy:
Threshold: 0.0000, Number of Features: 37, Accuracy: 0.6393
Threshold: 0.0156, Number of Features: 36, Accuracy: 0.6289
Threshold: 0.0190, Number of Features: 35, Accuracy: 0.6367
Threshold: 0.0192, Number of Features: 34, Accuracy: 0.6367
Threshold: 0.0208, Number of Features: 33, Accuracy: 0.6393
Threshold: 0.0209, Number of Features: 32, Accuracy: 0.6384
Threshold: 0.0211, Number of Features: 31, Accuracy: 0.6393
Threshold: 0.0220, Number of Features: 30, Accuracy: 0.6349
Threshold: 0.0221, Number of Features: 29, Accuracy: 0.6375
Threshold: 0.0222, Number of Features: 28, Accuracy: 0.6367
Threshold: 0.0222, Number of Features: 27, Accuracy: 0.6367
Threshold: 0.0225, Number of Features: 26, Accuracy: 0.6324
Threshold: 0.0225, Number of Features: 25, Accuracy: 0.6298
Threshold: 0.0228, Number of Features: 24, Accuracy: 0.6341
Threshold: 0.0230, Number of Features: 23, Accuracy: 0.6349
Threshold: 0.0232, Number of Feat

In [54]:
threshold = 0.0274 # Your specified threshold

# Select features using threshold
selected_features = X_train.columns[importances >= threshold]

# Print selected feature names and their count
selected_feature_names = list(selected_features)
number_of_features = len(selected_feature_names)

print("Selected Feature Names:", selected_feature_names)
print("Number of Selected Features:", number_of_features)


Selected Feature Names: [3, 8, 10, 12, 20, 34]
Number of Selected Features: 6


In [131]:
Index(['Volume', 'volume_pct_change', 'vol_ma_5', 'vol_ma_20', 'vol_ma_50',
       'vol_ma_200', 'ma_5', 'ma_10', 'ma_20', 'ma_50', 'ma_200',
       'volume_oscillator', 'relative_volume', 'vol_relative_to_ma', 'pvt',
       'SPY_VIX_ratio', 'rsi', 'bollinger_upper', 'bollinger_lower', 'cci',
       '10yr_bond_yield_change', 'yield_spread', 'atr', '%K', '%D',
       'SPY_pct_change', 'IWM_pct_change', 'SPY_IWM_ratio', 'QQQ_pct_change',
       'SPY_QQQ_ratio', 'DIA_pct_change', 'SPY_DIA_ratio', 'DXY_pct_change',
       'obv', 'momentum_5d', 'volatility_252d', 'momentum_volatility'],
      dtype='object')

NameError: name 'Index' is not defined

In [55]:
import yfinance as yf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score
from sklearn.ensemble import RandomForestClassifier

# Ensure this file exists in your directory
list1 = {"SPY"}
# Assuming Stock class and list1 are defined as before

# Lists to store combined features and labels for all stocks
all_features_rf = []
all_labels_rf = []

for tic in list1:
    # Download and preprocess data for each ticker in the list
    yf_data = yf.download(tic, start=start, end=end)
    
    # Skip if data is empty
    if yf_data.empty:
        continue

    stock = Stock(yf_data)
    stock.factor()        # Feature creation and preprocessing
    stock.standardize()   # Standardizing the data
    stock.normalize() 

    # Append the features and labels to the respective lists
    all_features_rf.append(pd.DataFrame(stock.data))
    all_labels_rf.extend(stock.label)

# Concatenating all features and labels into single datasets
X = pd.concat(all_features_rf)
y = pd.Series(all_labels_rf)

# Set a fixed random state for reproducibility
fixed_random_state = 42

# Splitting the dataset into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=fixed_random_state,shuffle=False)

rf_model = RandomForestClassifier(
    n_estimators=100,    # Number of trees
    max_depth=20,        # Maximum depth of each tree
    min_samples_split=2, # Minimum number of samples required to split an internal node
    min_samples_leaf=1,  # Minimum number of samples required to be at a leaf node
    max_features='sqrt', # Number of features to consider when looking for the best split
    bootstrap=True,      # Use bootstrap sampling
    random_state=fixed_random_state  # Ensures reproducibility
)

# Now you can train this model on your training data
rf_model.fit(X_train, y_train)

# And proceed with making predictions and evaluating the model as planned
pred_y_rf = rf_model.predict(X_test)

# Calculating accuracy and precision for the combined dataset
accuracy_rf = accuracy_score(y_test, pred_y_rf)
precision_rf = precision_score(y_test, pred_y_rf)
print("accuracy_rf: ", accuracy_rf)
print("precision_rf: ", precision_rf)


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
Index(['Volume', 'volume_pct_change', 'vol_ma_5', 'vol_ma_20', 'vol_ma_50',
       'vol_ma_200', 'ma_5'

In [23]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {
    'n_estimators': [100, 150, 200],  # Number of trees in the forest
    'max_depth': [8, 10, 12],  # Maximum depth of the tree
    'min_samples_split': [2, 4, 6],  # Minimum number of samples required to split a node
    'min_samples_leaf': [1, 2, 4],  # Minimum number of samples required at each leaf node
    'bootstrap': [True, False],  # Method of selecting samples for training each tree
}
# Assuming you have a features matrix X and a target vector y

# Initialize the random forest classifier
rf_model = RandomForestClassifier(random_state=fixed_random_state)

# Initialize GridSearchCV
grid_search = GridSearchCV(estimator=rf_model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2)

# Fit the grid search to the data
grid_search.fit(X, y)
best_parameters = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_parameters)
print("Best Score:", best_score)
best_estimator = grid_search.best_estimator_


Fitting 5 folds for each of 162 candidates, totalling 810 fits


Fatal Python error: init_sys_streams: can't initialize sys standard streams
Python runtime state: core initializedFatal Python error: init_sys_streams: can't initialize sys standard streams
Python runtime state: core initialized

OSError: [Errno 9] Bad file descriptor

OSErrorCurrent thread 0x: [Errno 9] Bad file descriptor

Current thread 0x00007ff85de36700 (most recent call first):
  <no Python frame>
00007ff85de36700 (most recent call first):
  <no Python frame>
Fatal Python error: init_sys_streams: can't initialize sys standard streams
Python runtime state: core initialized
OSError: [Errno 9] Bad file descriptor

Current thread 0x00007ff85de36700 (most recent call first):
  <no Python frame>
Fatal Python error: init_sys_streams: can't initialize sys standard streams
Python runtime state: core initialized
OSError: [Errno 9] Bad file descriptor

Current thread 0x00007ff85de36700 (most recent call first):
  <no Python frame>
Fatal Python error: init_sys_streams: can't initialize sys s

BrokenProcessPool: A task has failed to un-serialize. Please ensure that the arguments of the function are all picklable.

In [12]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Assuming X_train, X_test, y_train, y_test are already defined and preprocessed

# Initialize the Random Forest classifier
rf_model = RandomForestClassifier(random_state=fixed_random_state)

# Dictionary to store results
rf_results = {}

# Get feature importances from a previously fitted model (fit the model if not done yet)
rf_model.fit(X_train, y_train)
importances = rf_model.feature_importances_

# Get different thresholds for feature importance
thresholds = np.sort(np.unique(importances))

# Loop over the thresholds to find the best number of features based on accuracy
for thresh in thresholds:
    # Select features using threshold
    selection = X_train.columns[importances >= thresh]
    
    # Select training and testing data based on selected features
    select_X_train = X_train[selection]
    select_X_test = X_test[selection]
    
    # Train model with selected features
    selection_model = best_estimator
    selection_model.fit(select_X_train, y_train)
    
    # Evaluate model
    select_pred_y = selection_model.predict(select_X_test)
    select_accuracy = accuracy_score(y_test, select_pred_y)
    
    # Store results
    rf_results[thresh] = {'features': len(selection), 'accuracy': select_accuracy}

# Find the threshold with the highest accuracy
highest_accuracy = max(rf_results.items(), key=lambda x: x[1]['accuracy'])

# Print results
print("All Thresholds with Corresponding Number of Features and Accuracy:")
for thresh, data in rf_results.items():
    print(f"Threshold: {thresh:.4f}, Number of Features: {data['features']}, Accuracy: {data['accuracy']:.4f}")

print("\nHighest Accuracy:")
print(f"Threshold: {highest_accuracy[0]:.4f}, Number of Features: {highest_accuracy[1]['features']}, Accuracy: {highest_accuracy[1]['accuracy']:.4f}")



All Thresholds with Corresponding Number of Features and Accuracy:
Threshold: 0.0155, Number of Features: 38, Accuracy: 0.4784
Threshold: 0.0156, Number of Features: 37, Accuracy: 0.5038
Threshold: 0.0167, Number of Features: 36, Accuracy: 0.5588
Threshold: 0.0169, Number of Features: 35, Accuracy: 0.5131
Threshold: 0.0176, Number of Features: 34, Accuracy: 0.5529
Threshold: 0.0184, Number of Features: 33, Accuracy: 0.5775
Threshold: 0.0184, Number of Features: 32, Accuracy: 0.6088
Threshold: 0.0197, Number of Features: 31, Accuracy: 0.6071
Threshold: 0.0198, Number of Features: 30, Accuracy: 0.6190
Threshold: 0.0208, Number of Features: 29, Accuracy: 0.6063
Threshold: 0.0211, Number of Features: 28, Accuracy: 0.6113
Threshold: 0.0213, Number of Features: 27, Accuracy: 0.6147
Threshold: 0.0216, Number of Features: 26, Accuracy: 0.6046
Threshold: 0.0219, Number of Features: 25, Accuracy: 0.6249
Threshold: 0.0219, Number of Features: 24, Accuracy: 0.6071
Threshold: 0.0226, Number of Feat

detail search

In [58]:
from hyperopt import hp, tpe, fmin, Trials, STATUS_OK
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import TimeSeriesSplit


# Define the space of hyperparameters to search
space = {
    'n_estimators': hp.quniform('n_estimators', 50, 300, 1),
    'max_depth': hp.choice('max_depth', [None] + list(range(3, 25))),
    'min_samples_split': hp.quniform('min_samples_split', 2, 20, 1),
    'min_samples_leaf': hp.quniform('min_samples_leaf', 1, 5, 1),
    'bootstrap': hp.choice('bootstrap', [True, False]),
    'max_features': 20
}

# Objective function to minimize
def objective(params):
    # Convert float to int for certain parameters
    params['n_estimators'] = int(params['n_estimators'])
    params['min_samples_split'] = int(params['min_samples_split'])
    params['min_samples_leaf'] = int(params['min_samples_leaf'])

    rf_model = RandomForestClassifier(**params, random_state=fixed_random_state)
    
    # Use TimeSeriesSplit for cross-validation
    tscv = TimeSeriesSplit(n_splits=5)
    best_score = cross_val_score(rf_model, X, y, scoring='accuracy', cv=tscv).mean()
    return {'loss': -best_score, 'status': STATUS_OK}


# Run the algorithm
trials = Trials()
best = fmin(fn=objective,
            space=space,
            algo=tpe.suggest,
            max_evals=100,
            trials=trials)

depth_options = [None] + list(range(5, 31))  # Combined list of options for max_depth
best_params = {
    'n_estimators': int(best['n_estimators']),
    'max_depth': depth_options[int(best['max_depth'])],
    'min_samples_split': int(best['min_samples_split']),
    'min_samples_leaf': int(best['min_samples_leaf']),
    'bootstrap': [True, False][int(best['bootstrap'])]
}

print("Best Parameters:", best_params)

100%|████| 100/100 [1:05:00<00:00, 39.01s/trial, best loss: -0.6272066458982346]
Best Parameters: {'n_estimators': 204, 'max_depth': 5, 'min_samples_split': 11, 'min_samples_leaf': 5, 'bootstrap': True}


In [57]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import numpy as np

# Assuming X_train, X_test, y_train, y_test are already defined and preprocessed

# Initialize the Random Forest classifier with the best parameters
best_params = {'n_estimators': 189, 'max_depth': 5, 'min_samples_split': 13, 'min_samples_leaf': 1, 'bootstrap': True,
               'random_state': fixed_random_state}

best_rf_model = RandomForestClassifier(**best_params)

# Dictionary to store results
rf_results = {}

# Get feature importances from a previously fitted model
best_rf_model.fit(X_train, y_train)
importances = best_rf_model.feature_importances_

# Get different thresholds for feature importance
thresholds = np.sort(np.unique(importances))

# Loop over the thresholds to find the best number of features based on accuracy
for thresh in thresholds:
    # Select features using threshold
    selection = X_train.columns[importances >= thresh]
    
    # Select training and testing data based on selected features
    select_X_train = X_train[selection]
    select_X_test = X_test[selection]
    
    # Train model with selected features
    selection_model = RandomForestClassifier(**best_params)
    selection_model.fit(select_X_train, y_train)
    
    # Evaluate model
    select_pred_y = selection_model.predict(select_X_test)
    select_accuracy = accuracy_score(y_test, select_pred_y)
    
    # Store results
    rf_results[thresh] = {'features': len(selection), 'accuracy': select_accuracy}

# Find the threshold with the highest accuracy
highest_accuracy = max(rf_results.items(), key=lambda x: x[1]['accuracy'])

# Print results
print("All Thresholds with Corresponding Number of Features and Accuracy:")
for thresh, data in rf_results.items():
    print(f"Threshold: {thresh:.4f}, Number of Features: {data['features']}, Accuracy: {data['accuracy']:.4f}")

print("\nHighest Accuracy:")
print(f"Threshold: {highest_accuracy[0]:.4f}, Number of Features: {highest_accuracy[1]['features']}, Accuracy: {highest_accuracy[1]['accuracy']:.4f}")


All Thresholds with Corresponding Number of Features and Accuracy:
Threshold: 0.0080, Number of Features: 37, Accuracy: 0.6453
Threshold: 0.0097, Number of Features: 36, Accuracy: 0.6332
Threshold: 0.0099, Number of Features: 35, Accuracy: 0.6358
Threshold: 0.0100, Number of Features: 34, Accuracy: 0.6358
Threshold: 0.0111, Number of Features: 33, Accuracy: 0.6324
Threshold: 0.0112, Number of Features: 32, Accuracy: 0.6324
Threshold: 0.0112, Number of Features: 31, Accuracy: 0.6367
Threshold: 0.0117, Number of Features: 30, Accuracy: 0.6341
Threshold: 0.0127, Number of Features: 29, Accuracy: 0.6332
Threshold: 0.0127, Number of Features: 28, Accuracy: 0.6298
Threshold: 0.0128, Number of Features: 27, Accuracy: 0.6358
Threshold: 0.0130, Number of Features: 26, Accuracy: 0.6263
Threshold: 0.0131, Number of Features: 25, Accuracy: 0.6384
Threshold: 0.0131, Number of Features: 24, Accuracy: 0.6289
Threshold: 0.0138, Number of Features: 23, Accuracy: 0.6298
Threshold: 0.0145, Number of Feat

In [30]:
threshold = 0.0220  # Your specified threshold

# Select features using threshold
selected_features = X_train.columns[importances >= threshold]

# Print selected feature names and their count
selected_feature_names = list(selected_features)
number_of_features = len(selected_feature_names)

print("Selected Feature Names:", selected_feature_names)
print("Number of Selected Features:", number_of_features)


Selected Feature Names: [1, 12, 13, 16, 32, 34]
Number of Selected Features: 6


In [ ]:
Index(['Volume', 'volume_pct_change', 'vol_ma_5', 'vol_ma_20', 'vol_ma_50',
       'vol_ma_200', 'ma_5', 'ma_10', 'ma_20', 'ma_50', 'ma_200',
       'volume_oscillator', 'relative_volume', 'vol_relative_to_ma', 'pvt',
       'SPY_VIX_ratio', 'rsi', 'bollinger_upper', 'bollinger_lower', 'cci',
       '10yr_bond_yield_change', 'yield_spread', 'atr', '%K', '%D',
       'SPY_pct_change', 'IWM_pct_change', 'SPY_IWM_ratio', 'QQQ_pct_change',
       'SPY_QQQ_ratio', 'DIA_pct_change', 'SPY_DIA_ratio', 'DXY_pct_change',
       'obv', 'momentum_5d', 'volatility_252d', 'momentum_volatility'],
      dtype='object')

 Next part is for group of tickers, you don't need to bother them now.

In [66]:
ticker = pd.read_csv("tickers.csv")  # Ensure this file exists in your directory
list1 = list(ticker["Ticker"])
# Custom data loading class for backtrader
class PredictionsData(bt.feeds.PandasData):
    lines = ('predictions',)
    params = (('predictions', -1),)

def get_stock_predictions(ticker, model):
    yf_data = yf.download(ticker, start="2000-01-01", end="2021-11-12")
    if yf_data.empty:
        return None

    # Preprocess the data
    stock = Stock(yf_data.copy())
    stock.factor()
    stock.standardize()
    stock.normalize()

    # Splitting the processed data
    X = pd.DataFrame(stock.data)
    _, X_test = train_test_split(X, test_size=0.4, random_state=42)

    # Generate predictions for the test set
    test_predictions = model.predict(X_test)

    # Store predictions in a DataFrame with the same index as the original data
    predictions_series = pd.Series(index=yf_data.index)
    predictions_series[X_test.index] = test_predictions
    yf_data['predictions'] = predictions_series

    return yf_data

# Backtrader strategy class
class RFStrategy(bt.Strategy):
    def __init__(self):
        self.predicted = self.datas[0].predictions

    def next(self):
        if not self.position:
            if self.predicted[0] == 1 and self.broker.get_cash() > 100:
                self.buy()
        elif self.predicted[0] == 0 and self.getposition().size > 0:
            self.sell()

FileNotFoundError: [Errno 2] No such file or directory: 'tickers.csv'

In [43]:
import pandas as pd

evaluation_metrics = {
    "rf_model": {
        "Accuracy": accuracy_rf, 
        "Precision": precision_rf
    },
    "xgb_model": {
        "Accuracy": accuracy, 
        "Precision": precision
    }
}

models = [rf_model, xgb_model]
model_names = ['rf_model', 'xgb_model']
top_tickers = {}  # Dictionary to store top two tickers for each model
ticker_data = pd.read_csv("tickers.csv")  # Ensure this file exists
list1 = list(ticker_data["Ticker"])

for model, model_name in zip(models, model_names):
    final_values = {}
    stock_predictions = {}
    for ticker in list1:
        processed_data = get_stock_predictions(ticker, model)
        if processed_data is not None:
            stock_predictions[ticker] = processed_data

    # Running backtest and storing final portfolio values
    for ticker, data in stock_predictions.items():
        cerebro = bt.Cerebro()
        cerebro.addstrategy(RFStrategy)
        data_feed = PredictionsData(dataname=data)
        cerebro.adddata(data_feed)
        cerebro.broker.set_cash(10000)
        cerebro.broker.setcommission(commission=0.001)
        cerebro.run()
        final_val = cerebro.broker.getvalue()
        final_values[ticker] = final_val

    # Identify top two tickers
    top_two = sorted(final_values, key=final_values.get, reverse=True)[:2]
    top_tickers[model_name] = top_two

combined_output_df = pd.DataFrame({
    "Model": ['Random Forest', 'XGBoost'],
    "Top_Ticker_1": [top_tickers[model][0] for model in model_names],
    "Top_Ticker_2": [top_tickers[model][1] for model in model_names],
    "Accuracy_1": [evaluation_metrics[model]["Accuracy"] for model in model_names],
    "Precision_1": [evaluation_metrics[model]["Precision"] for model in model_names],
    "Accuracy_2": [evaluation_metrics[model]["Accuracy"] for model in model_names],
    "Precision_2": [evaluation_metrics[model]["Precision"] for model in model_names]
})

# # Export to CSV
combined_output_df.to_csv('small_universe_results.csv', index=False)

NameError: name 'accuracy' is not defined

In [8]:
def generate_report_for_ticker(ticker, model, strategy_class, stock_predictions):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy_class)
    data_feed = PredictionsData(dataname=stock_predictions[ticker])
    cerebro.adddata(data_feed)
    cerebro.broker.set_cash(10000)
    cerebro.broker.setcommission(commission=0.001)
    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='time_return')
    strat = cerebro.run()
    daily_returns = strat[0].analyzers.time_return.get_analysis()
    returns_series = pd.Series(daily_returns)
    returns_series.index = pd.to_datetime(returns_series.index)

    # Generate and save reports
    qs.reports.html(returns_series, output=f'quantstats_{ticker}_{model}.html')

# Generate reports for top tickers of each model
for model_name, tickers in top_tickers.items():
    for ticker in tickers:
        generate_report_for_ticker(ticker, model_name, RFStrategy, stock_predictions)

In [9]:
def process_stock_data(data):
    if data.empty:
        return None, None

    stock = Stock(data)
    stock.factor()  # Feature creation and preprocessing
    stock.standardize()  # Standardizing the data

    # Check if data is still non-empty after standardization
    if stock.data.size == 0:  # Use .size for NumPy arrays
        return None, None

    stock.normalize()  # Normalizing the data

    X = pd.DataFrame(stock.data)
    y = pd.Series(stock.label)

    return X, y

In [10]:
def fetch_data(ticker):
    try:
        data = yf.download(ticker,start="2000-01-01", end="2021-11-12")
        if data.empty:
            return None
        else:
            return data
    except Exception as e:
        return None

In [ ]:
ticker_df = pd.read_csv("tickers_nasd.csv")  
tickers = list(ticker_df["Symbol"])
results_rf = {}
results_xgb = {}

# Initialize dictionaries to store results
accuracy_results_rf = {}
accuracy_results_xgb = {}

# Initialize a dictionary to store combined accuracies
combined_accuracy_results = {}

for ticker in tickers:
    fetched_data = fetch_data(ticker)
    if fetched_data is None or fetched_data.empty:
        continue

    X, y = process_stock_data(fetched_data)
    if X is None or y is None or X.empty or len(y) < 2:
        continue

    # Ensure there's enough data to split
    if len(X) > 1:
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.4, random_state=42)

        # Random Forest Model
        rf_model.fit(X_train, y_train)
        accuracy_rf = accuracy_score(y_test, rf_model.predict(X_test))
        accuracy_results_rf[ticker] = accuracy_rf  # Store RF accuracy

        # XGBoost Model
        xgb_model.fit(X_train, y_train)
        accuracy_xgb = accuracy_score(y_test, xgb_model.predict(X_test))
        accuracy_results_xgb[ticker] = accuracy_xgb  # Store XGB accuracy

        # Combine accuracies (here, taking the average)
        combined_accuracy = (accuracy_rf + accuracy_xgb) / 2
        combined_accuracy_results[ticker] = combined_accuracy



# Sort and select top 10 tickers based on combined model accuracies
top_10_tickers_combined = sorted(combined_accuracy_results, key=combined_accuracy_results.get, reverse=True)[:10]

In [12]:
# Sort and select top 10 tickers based on combined model accuracies
top_10_tickers_combined = sorted(combined_accuracy_results, key=combined_accuracy_results.get, reverse=True)[:10]

In [13]:
# Create a DataFrame to display the tickers along with accuracies from both models
top_10_df = pd.DataFrame({
    'Ticker': top_10_tickers_combined,
    'Combined_Accuracy': [combined_accuracy_results.get(ticker, None) for ticker in top_10_tickers_combined],
    'RF_Accuracy': [accuracy_results_rf.get(ticker, None) for ticker in top_10_tickers_combined],
    'XGB_Accuracy': [accuracy_results_xgb.get(ticker, None) for ticker in top_10_tickers_combined]
})


# Convert the DataFrame to a CSV string and write to a file
csv_data = top_10_df.to_csv(index=False)
with open('top_10_combined_accuracy.csv', 'w') as file:
    file.write(csv_data)

In [14]:
stock_predictions = {}
for ticker in top_10_tickers_combined:
    predictions = get_stock_predictions(ticker, rf_model)  # Replace 'your_model' with the actual model
    if predictions is not None:
        stock_predictions[ticker] = predictions


[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


In [15]:
for ticker in top_10_tickers_combined:
    cerebro = bt.Cerebro()
    cerebro.addstrategy(RFStrategy)
    data_feed = PredictionsData(dataname=stock_predictions[ticker])
    cerebro.adddata(data_feed)
    cerebro.broker.set_cash(10000)
    cerebro.broker.setcommission(commission=0.001)
    cerebro.addanalyzer(bt.analyzers.TimeReturn, _name='time_return')
    strat = cerebro.run()
    daily_returns = strat[0].analyzers.time_return.get_analysis()
    returns_series = pd.Series(daily_returns)
    returns_series.index = pd.to_datetime(returns_series.index)

    # Generate and save reports
    qs.reports.html(returns_series, output=f'quantstats_{ticker}.')